## Auto-Complete client messages for Kraken Interview Using N-Gram model

In [229]:
import math
import random
import numpy as np
import pandas as pd
import nltk
import sqlite3
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import re
nltk.data.path.append('.')

In [201]:


cnx = sqlite3.connect('reddit.db')

df = pd.read_sql_query("SELECT * FROM comment", cnx)


In [202]:
df

,id,author,body,subreddit
0,c2uik3z,herrrro,my comment will be buried but what he heck \n ...,AskReddit
1,c7ww7cy,DrCramfingers,"In the seventh grade, I played lacrosse in my ...",AskReddit
2,c9hy6n8,magnetsforbreakfast,"Edit: apparently I misread the question, so en...",AskReddit
3,c2c5f9d,protosynk,"In highschool, my friends and I used to carpoo...",AskReddit
4,cq8wgxd,banana-mouse,I'm afraid of planes which limits where I can ...,solotravel
...,...,...,...,...
10326,cbpr7gu,MyNameisTingles,I tried this week really hard to get into Hit...,Games
10327,ceq9f0s,esseestpercipi,"A lot of people are advocating for no contact,...",AskWomen
10328,cbnjf8e,Razvedka,It goes further than that. The FBI was alerted...,technology
10329,cizph4k,FearlessBurrito,"Okay so, you know how water parks sometimes ha...",fatlogic


In [203]:
df["body"][0]

'my comment will be buried but what he heck \n i was probably 8 or 9 at the time, trampolines were the best shit in the world. one day doing flips i lost my sense of reality or something and went head first strait into the ground with my eyes squeezing tight knowing my fate is at hand...i open my eyes and my head is inches above the ground and im dangling there. my legs caught the space between the springs luckily, and i knew i had nothing to do with it. i wasnt trying to save myself but i believe God was watchin out for me. thanks Jesus, I love you reddit :) dont downvote me lol \n TLDR: flying spaghetti monster lass-owed me inches from certain death!! \n'

In [204]:
data = df["body"].str.cat(sep=' ')

In [205]:
def load_bad_words(file_path):
    """
    Load bad words from a text file into a list.
    
    Args:
        file_path (str): Path to the text file containing bad words.
    
    Returns:
        list: A list of bad words.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        bad_words = [line.strip() for line in file if line.strip()]
    return bad_words


In [206]:
bad_words = load_bad_words('Forbiden_words.txt')

df

,id,author,body,subreddit
0,c2uik3z,herrrro,my comment will be buried but what he heck \n ...,AskReddit
1,c7ww7cy,DrCramfingers,"In the seventh grade, I played lacrosse in my ...",AskReddit
2,c9hy6n8,magnetsforbreakfast,"Edit: apparently I misread the question, so en...",AskReddit
3,c2c5f9d,protosynk,"In highschool, my friends and I used to carpoo...",AskReddit
4,cq8wgxd,banana-mouse,I'm afraid of planes which limits where I can ...,solotravel
...,...,...,...,...
10326,cbpr7gu,MyNameisTingles,I tried this week really hard to get into Hit...,Games
10327,ceq9f0s,esseestpercipi,"A lot of people are advocating for no contact,...",AskWomen
10328,cbnjf8e,Razvedka,It goes further than that. The FBI was alerted...,technology
10329,cizph4k,FearlessBurrito,"Okay so, you know how water parks sometimes ha...",fatlogic


### Split data into sentences.



In [207]:
# Function to remove emojis and special characters from text
def remove_emojis_and_special_characters(text):
    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Remove extra spaces
    text = re.sub(' +', ' ', text)
    
    return text


### Word-level tokenization

In [209]:
nltk.download('punkt_tab')

def tokenize_sentences(sentences):
    """
    Tokenize sentences into tokens (words)
    
    Args:
        sentences: List of strings
    
    Returns:
        List of lists of tokens
    """

    tokenized_sentences = []

    

    for sentence in sentences:
        
 
        sentence = sentence.lower()

        tokenized = nltk.word_tokenize(sentence)

        tokenized_sentences.append(tokenized)
    
    return tokenized_sentences

[nltk_data] Downloading package punkt_tab to /home/hamdi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


We start by loading a list of bad words from a file ("found online"), which helps us filter out inappropriate content. Our preprocessing steps include removing emojis and special characters to standardize the text, ensuring it is free from unnecessary noise. We eliminate sentences containing bad words to maintain the appropriateness of the data, and we convert the remaining text to lowercase for consistency.

In [211]:

def load_bad_words(file_path):
    """
    Load bad words from a text file into a list.
    
    Args:
        file_path (str): Path to the text file containing bad words.
    
    Returns:
        list: A list of bad words.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        bad_words = [line.strip() for line in file if line.strip()]
    return bad_words

def remove_emojis_and_special_characters(text):
    """
    Remove emojis and special characters from the text.
    
    Args:
        text (str): The input text from which to remove emojis and special characters.
    
    Returns:
        str: The cleaned text.
    """
    import re
    return re.sub(r'[^\w\s]', '', text)  # Remove all non-word characters except spaces

def remove_sentence_bad_words(texts, bad_words):
    """
    Remove strings from the input list that contain bad words.
    
    Args:
        texts (list): A list of strings to check for bad words.
        bad_words (list): A list of bad words to filter out.
    
    Returns:
        tuple: A tuple containing the cleaned list and the count of eliminated rows.
    """

    pattern = r'\b(?:' + '|'.join(bad_words) + r')\b'
    

    filtered_texts = [text for text in texts if not pd.Series(text).str.contains(pattern, case=False, na=False).any()]
    
    eliminated_count = len(texts) - len(filtered_texts)
    
    return filtered_texts, eliminated_count

def preprocess_pipeline(data, bad_words) -> 'list':
    """
    Preprocess the input text data by cleaning and tokenizing sentences.
    
    Args:
        data (str): The input text data to preprocess.
        bad_words (list): A list of bad words to filter out from the sentences.
    
    Returns:
        list: A list of cleaned and tokenized sentences.
    """

    sentences = data.split('\n')
    

    for i in range(len(sentences)):
        sentences[i] = remove_emojis_and_special_characters(sentences[i])
    

    sentences = [s.strip() for s in sentences]
    

    sentences = [s for s in sentences if len(s) > 0]
    

    cleaned_sentences, _ = remove_sentence_bad_words(sentences, bad_words)
    

    tokenized = [sentence.lower() for sentence in cleaned_sentences]
    
    return tokenized





In [212]:
bad_words = load_bad_words('Forbiden_words.txt')

# Example data to preprocess
test_data = """Hello, world!
This is a test sentence.
Let's clean this up! 😊
This sentence contains a badword ."""

# Tokenize sentences
test_tokenized_sentences = preprocess_pipeline(test_data, bad_words)

# Output the results
print(test_tokenized_sentences)

['hello world', 'this is a test sentence', 'lets clean this up', 'this sentence contains a badword']


### Train-test Spliting dataset


In [213]:
tokenized_sentences = preprocess_pipeline(data,bad_words)

In [214]:
tokenized_sentences

['my comment will be buried but what he heck',
 'tldr flying spaghetti monster lassowed me inches from certain death',
 'in the seventh grade i played lacrosse in my citys league at practice one day we were doing one on ones and it came to be my turn to be the defender i took my place between the attacker and the goal and squared up as he ran toward me he began to dodge to my right and so i started turning to follow my right leg and rest of my body kept with the turn but my left leg had other plans i simply did not move it just kept it where it was haha as i turned my mcl snapped and i collapsed continued to practice on my nonfunctioning leg and finally went to the doc after two days after no improvement',
 'tldr tried to turn left leg said no mcl snapped like a taut rubber band',
 'edit apparently i misread the question so enjoy my story of food i enjoyed as a child but hate now',
 'when i was about 8 my mother who normally cooked dinner went on a trip to new york for a week i think m

In [216]:
tokenized_sentences[:10]

['my comment will be buried but what he heck',
 'tldr flying spaghetti monster lassowed me inches from certain death',
 'in the seventh grade i played lacrosse in my citys league at practice one day we were doing one on ones and it came to be my turn to be the defender i took my place between the attacker and the goal and squared up as he ran toward me he began to dodge to my right and so i started turning to follow my right leg and rest of my body kept with the turn but my left leg had other plans i simply did not move it just kept it where it was haha as i turned my mcl snapped and i collapsed continued to practice on my nonfunctioning leg and finally went to the doc after two days after no improvement',
 'tldr tried to turn left leg said no mcl snapped like a taut rubber band',
 'edit apparently i misread the question so enjoy my story of food i enjoyed as a child but hate now',
 'when i was about 8 my mother who normally cooked dinner went on a trip to new york for a week i think m

In [233]:
nltk.download('punkt_tab')

def tokenize_sentences(sentences):
    """
    Tokenize sentences into tokens (words)
    
    Args:
        sentences: List of strings
    
    Returns:
        List of lists of tokens
    """

    tokenized_sentences = []

    for sentence in sentences:

        sentence = sentence.lower()

        tokenized = nltk.word_tokenize(sentence)

        tokenized_sentences.append(tokenized)
    
    return tokenized_sentences

[nltk_data] Downloading package punkt_tab to /home/hamdi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [234]:
tokenized_sentences = tokenize_sentences(tokenized_sentences)

In [235]:
tokenized_sentences

[['my', 'comment', 'will', 'be', 'buried', 'but', 'what', 'he', 'heck'],
 ['tldr',
  'flying',
  'spaghetti',
  'monster',
  'lassowed',
  'me',
  'inches',
  'from',
  'certain',
  'death'],
 ['in',
  'the',
  'seventh',
  'grade',
  'i',
  'played',
  'lacrosse',
  'in',
  'my',
  'citys',
  'league',
  'at',
  'practice',
  'one',
  'day',
  'we',
  'were',
  'doing',
  'one',
  'on',
  'ones',
  'and',
  'it',
  'came',
  'to',
  'be',
  'my',
  'turn',
  'to',
  'be',
  'the',
  'defender',
  'i',
  'took',
  'my',
  'place',
  'between',
  'the',
  'attacker',
  'and',
  'the',
  'goal',
  'and',
  'squared',
  'up',
  'as',
  'he',
  'ran',
  'toward',
  'me',
  'he',
  'began',
  'to',
  'dodge',
  'to',
  'my',
  'right',
  'and',
  'so',
  'i',
  'started',
  'turning',
  'to',
  'follow',
  'my',
  'right',
  'leg',
  'and',
  'rest',
  'of',
  'my',
  'body',
  'kept',
  'with',
  'the',
  'turn',
  'but',
  'my',
  'left',
  'leg',
  'had',
  'other',
  'plans',
  'i',
  '

In [236]:

def count_words(tokenized_sentences):
    """
    Count the number of word appearence in the tokenized sentences
    
    Args:
        tokenized_sentences: List of lists of strings
    
    Returns:
        dict that maps word (str) to the frequency (int)
    """
        
    word_counts = {}
    for sentence in tokenized_sentences:

        for token in sentence: 
            if token not in word_counts.keys(): 
                word_counts[token] = 1
            else:
                word_counts[token] += 1

    
    return word_counts

#### Handling 'Out of Vocabulary' words


In [237]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    """
    Find the words that appear N times or more
    
    Args:
        tokenized_sentences: List of lists of sentences
        count_threshold: minimum number of occurrences for a word to be in the closed vocabulary.
    
    Returns:
        List of words that appear N times or more
    """
    closed_vocab = []

    word_counts = count_words(tokenized_sentences)

    for word, cnt in word_counts.items():

        if cnt >= count_threshold:
            closed_vocab.append(word)
    
    return closed_vocab

The words that appear `count_threshold` times or more are in the closed vocabulary. 
- All other words are regarded as `unknown`.
- We replace words not in the closed vocabulary with the token `<oov>`.

In [238]:

def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<oov>"):
    """
    Replace words not in the given vocabulary with '<oov>' token.
    
    Args:
        tokenized_sentences: List of lists of strings
        vocabulary: List of strings that we will use
        unknown_token: A string representing unknown (out-of-vocabulary) words
    
    Returns:
        List of lists of strings, with words not in the vocabulary replaced
    """
    

    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []

    for sentence in tokenized_sentences:

        replaced_sentence = []
        for token in sentence: 
            if token in vocabulary: 

                replaced_sentence.append(token)
            else:

                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)
        
    return replaced_tokenized_sentences

In [239]:

def preprocess_data(train_data, count_threshold):
    """
    Preprocess data, i.e.,
        - Find tokens that appear at least N times in the training data.
        - Replace tokens that appear less than N times by "<oov>" both for training and test data.        
    Args:
        train_data: List of lists of strings.
        count_threshold: Words whose count is less than this are 
                      treated as unknown.
    
    Returns:
        Tuple of
        - training data with low frequent words replaced by "<oov>"
        - vocabulary of words that appear n times or more in the training data
    """
    vocabulary = get_words_with_nplus_frequency(train_data,count_threshold)
    
    train_data_replaced = replace_oov_words_by_unk(train_data,vocabulary)

    return train_data_replaced, vocabulary

### Preprocess the train and test data
Run the cell below to complete the preprocessing both for training and test sets.

In [240]:
minimum_freq = 2
train_data_processed, vocabulary = preprocess_data(tokenized_sentences, minimum_freq)

<a name='2'></a>
## First solution (Basic one): n-gram based language models

An **n-gram language model** has both advantages and disadvantages, as well as core principles that guide its functionality:

### **Principle:**
- The model assumes that the probability of a word depends only on the previous 'n' words, ignoring longer context or sentence structure.
- It estimates conditional probabilities by counting the occurrences of word sequences (n-grams) in training data.
- The formula for estimating probability is based on the frequency of the n-gram and (n+1)-gram counts.

### **Advantages:**
1. **Simplicity**: N-gram models are easy to understand and implement, requiring only word counts to estimate probabilities.
2. **Efficiency**: They can be trained quickly with relatively low computational resources compared to more complex models like neural networks.
3. **Scalability**: N-gram models can handle large corpora efficiently as they are based on frequency counts.
4. **Adaptability**: They can capture some level of local context (within the n preceding words), providing a practical balance between context and computation.

### **Disadvantages:**
1. **Data Sparsity**: As n increases, the number of possible n-grams grows exponentially, leading to sparse data. Many n-grams may never appear in the training data, making probability estimation unreliable.
2. **Limited Context**: N-gram models have a fixed context window (the previous 'n' words). This limits their ability to capture long-range dependencies or deeper syntactic and semantic relationships in language.
3. **Zero Probability for Unseen Words**: Without smoothing, the model assigns zero probability to word sequences that are not seen in training, which can lead to poor performance on unseen data.
4. **Storage and Computational Cost**: For higher values of n, storing and computing n-gram probabilities can become expensive in terms of memory and processing time.

In summary, n-gram models are simple and efficient but struggle with sparsity and capturing long-term dependencies in language.

In [241]:

def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    """
    Count all n-grams in the data
    
    Args:
        data: List of lists of words
        n: number of words in a sequence
    
    Returns:
        A dictionary that maps a tuple of n-words to its frequency
    """
    
    n_grams = {}

    for sentence in data: 
        
        sentence = [start_token] * n + sentence + [end_token]
        sentence = tuple(sentence)
        m = len(sentence) if n == 1 else len(sentence) - 1
        for i in range(m): 
            n_gram = sentence[i:i+n]
            if n_gram in n_grams.keys(): 
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1

    return n_grams

### Estimate probabilities for all words

The function defined below loops over all words in vocabulary to calculate probabilities (i.e.,the probability of a word given the prior 'n' words using the n-gram counts.
) for all possible words.

In [242]:

def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    """
    Estimate the probabilities of next words using the n-gram counts with k-smoothing
    
    Args:
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary: List of words
        k: positive constant, smoothing parameter
    
    Returns:
        A dictionary mapping from next words to the probability.
    """
    
    previous_n_gram = tuple(previous_n_gram)
    
    vocabulary = vocabulary + ["<e>", "<oov>"]
    vocabulary_size = len(vocabulary)
    
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           vocabulary_size, k=k)
        probabilities[word] = probability

    return probabilities



def estimate_probability(word, previous_n_gram, 
                         n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    previous_n_gram = tuple(previous_n_gram)

    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
    denominator = previous_n_gram_count + k * vocabulary_size
    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0
    numerator = n_plus1_gram_count + k
    probability = numerator / denominator
    
    return probability

In [243]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):

    vocabulary = vocabulary + [ "<oov>"]

    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    col_index = {word:j for j, word in enumerate(vocabulary)}
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [244]:

def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    
    n = len(list(n_gram_counts.keys())[0]) 
    

    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    suggestion = None

    max_prob = 0

    for word, prob in probabilities.items():
        
        if start_with: 
            
            if not word.startswith(start_with):
                continue  
        if prob > max_prob: 
            suggestion = word          
            max_prob = prob
    
    return suggestion, max_prob

### Get multiple suggestions

The function defined below loop over varioud n-gram models to get multiple suggestions.

In [245]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [261]:
n_gram_counts_list

[{('<s>',): 47698,
  ('my',): 14370,
  ('comment',): 243,
  ('will',): 4811,
  ('be',): 11240,
  ('buried',): 41,
  ('but',): 12377,
  ('what',): 5531,
  ('he',): 6956,
  ('heck',): 38,
  ('<e>',): 47698,
  ('tldr',): 8546,
  ('flying',): 75,
  ('spaghetti',): 11,
  ('monster',): 78,
  ('<oov>',): 32918,
  ('me',): 7788,
  ('inches',): 55,
  ('from',): 6319,
  ('certain',): 401,
  ('death',): 294,
  ('in',): 25948,
  ('the',): 79831,
  ('seventh',): 6,
  ('grade',): 171,
  ('i',): 39927,
  ('played',): 449,
  ('lacrosse',): 7,
  ('citys',): 4,
  ('league',): 162,
  ('at',): 8507,
  ('practice',): 239,
  ('one',): 5775,
  ('day',): 1868,
  ('we',): 7222,
  ('were',): 4107,
  ('doing',): 1128,
  ('on',): 12961,
  ('ones',): 468,
  ('and',): 49444,
  ('it',): 23326,
  ('came',): 839,
  ('to',): 53389,
  ('turn',): 517,
  ('defender',): 19,
  ('took',): 781,
  ('place',): 819,
  ('between',): 823,
  ('attacker',): 10,
  ('goal',): 204,
  ('squared',): 5,
  ('up',): 6110,
  ('as',): 11162,


In [277]:

import pickle

n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)


# After counting your n-grams:
with open('Server/ngram_counts.pkl', 'wb') as file:
    pickle.dump(n_gram_counts_list, file)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [278]:
# Load N-gram model from file
import pickle
with open('Server/ngram_counts.pkl', 'rb') as f:
    n_gram_counts_list = pickle.load(f)

In [279]:
previous_tokens = ["I", "am", "going", "to"]
tmp_suggest = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest)

The previous words are ['I', 'am', 'going', 'to'], the suggestions are:


[('the', 0.04239821336540114),
 ('be', 0.007060224566178972),
 ('be', 8.839127872716559e-05),
 ('my', 2.9475918174851147e-05)]